# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline


%matplotlib inline
import seaborn as sns
import folium 

In [2]:
#importing datasets
df = pd.read_csv('../Traffic_Crashes_-_Crashes.csv')
df_people = pd.read_csv('../Traffic_Crashes_-_People.csv')
df_vehicles = pd.read_csv('../Traffic_Crashes_-_Vehicles.csv')
df.head()

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12,13,20,21,22,23,24,25,26,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,JC343143,NaN,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,JA329216,NaN,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,JD292400,NaN,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,3.0,0.0,10,6,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008)
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,JD293602,NaN,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,...,0.0,0.0,3.0,0.0,1,7,7,41.802119,-87.622115,POINT (-87.622114914961 41.802118543011)
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,JD290451,NaN,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,14,4,7,NaN,NaN,NaN


In [3]:
df_vehicles.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,JD124535,01/22/2020 06:25:00 AM,1,DRIVER,NaN,796949.0,NaN,INFINITI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,04/13/2020 10:50:00 PM,2,DRIVER,NaN,827212.0,NaN,BUICK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567797 entries, 0 to 567796
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                567797 non-null  object 
 1   RD_NO                          563961 non-null  object 
 2   CRASH_DATE_EST_I               42926 non-null   object 
 3   CRASH_DATE                     567797 non-null  object 
 4   POSTED_SPEED_LIMIT             567797 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         567797 non-null  object 
 6   DEVICE_CONDITION               567797 non-null  object 
 7   WEATHER_CONDITION              567797 non-null  object 
 8   LIGHTING_CONDITION             567797 non-null  object 
 9   FIRST_CRASH_TYPE               567797 non-null  object 
 10  TRAFFICWAY_TYPE                567797 non-null  object 
 11  LANE_CNT                       198975 non-null  float64
 12  ALIGNMENT                     

In [5]:
df_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251211 entries, 0 to 1251210
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1251211 non-null  object 
 1   PERSON_TYPE            1251211 non-null  object 
 2   CRASH_RECORD_ID        1251211 non-null  object 
 3   RD_NO                  1242411 non-null  object 
 4   VEHICLE_ID             1226460 non-null  float64
 5   CRASH_DATE             1251211 non-null  object 
 6   SEAT_NO                255463 non-null   float64
 7   CITY                   918813 non-null   object 
 8   STATE                  929935 non-null   object 
 9   ZIPCODE                838905 non-null   object 
 10  SEX                    1232275 non-null  object 
 11  AGE                    890752 non-null   float64
 12  DRIVERS_LICENSE_STATE  737397 non-null   object 
 13  DRIVERS_LICENSE_CLASS  633054 non-null   object 
 14  SAFETY_EQUIPMENT  

In [6]:
df_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162795 entries, 0 to 1162794
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1162795 non-null  int64  
 1   CRASH_RECORD_ID           1162795 non-null  object 
 2   RD_NO                     1154843 non-null  object 
 3   CRASH_DATE                1162795 non-null  object 
 4   UNIT_NO                   1162795 non-null  int64  
 5   UNIT_TYPE                 1161127 non-null  object 
 6   NUM_PASSENGERS            173909 non-null   float64
 7   VEHICLE_ID                1136152 non-null  float64
 8   CMRC_VEH_I                21466 non-null    object 
 9   MAKE                      1136147 non-null  object 
 10  MODEL                     1136003 non-null  object 
 11  LIC_PLATE_STATE           1035993 non-null  object 
 12  VEHICLE_YEAR              950868 non-null   float64
 13  VEHICLE_DEFECT            1

In [7]:
df.shape

(567797, 49)

# Data Preparation

In [8]:
#Merging Datasets on the Crash_Record_ID Column
df_merge = pd.merge(df, df_vehicles, on='CRASH_RECORD_ID').reset_index()
df_merge_2 = pd.merge(df_merge, df_people, on='CRASH_RECORD_ID').reset_index()
#dropping dupllicates(basically only having one instance of the crashID on there)
df_dropped= df_merge_2.drop_duplicates(subset=['CRASH_RECORD_ID'], keep='first')
#Taking only necessary columns
columns = ['CRASH_RECORD_ID', 'RD_NO_x', 'CRASH_DATE_x', 'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME',  'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'RD_NO_y', 'UNIT_NO', 'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID_x', 'CMRC_VEH_I', 'MAKE', 'MODEL', 'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'TRAVEL_DIRECTION', 'MANEUVER', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'FIRST_CONTACT_POINT', 'PERSON_TYPE', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE',  'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'DRIVER_VISION','PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE', 'CELL_PHONE_USE']
df_comb = df_dropped[columns]

In [9]:
#Taking the Year from the date column
df_comb['date'] = pd.to_datetime(df_comb['CRASH_DATE_x'])
df_comb['Crash_year'] = df_comb['date'].apply(lambda date: date.year)
#only accidents with the driver
df_driver = df_comb[df_comb['PERSON_TYPE']=='DRIVER']
#necessary columns
new_columns = ['CRASH_RECORD_ID', 'CRASH_DATE_x', 'POSTED_SPEED_LIMIT',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE',
       'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE','NUM_UNITS',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT','CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'LATITUDE', 'LONGITUDE', 'MAKE', 'MODEL',
       'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE',
       'MANEUVER', 'OCCUPANT_CNT', 
       'FIRST_CONTACT_POINT', 'PERSON_TYPE','SEX',
       'AGE', 'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION','DRIVER_VISION',
       'PHYSICAL_CONDITION', 'Crash_year']

<ipython-input-9-6efbe8c7bb70>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comb['date'] = pd.to_datetime(df_comb['CRASH_DATE_x'])
<ipython-input-9-6efbe8c7bb70>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comb['Crash_year'] = df_comb['date'].apply(lambda date: date.year)


In [10]:
#going to drop columns with lots of missing values
pd.isnull(df_driver).sum().sort_values(ascending=False)

PEDPEDAL_LOCATION              544771
PEDPEDAL_VISIBILITY            544771
PEDPEDAL_ACTION                544771
CELL_PHONE_USE                 543952
WORKERS_PRESENT_I              543936
                                ...  
INJURIES_NON_INCAPACITATING         0
INJURIES_INCAPACITATING             0
INJURIES_FATAL                      0
INJURIES_TOTAL                      0
CRASH_RECORD_ID                     0
Length: 78, dtype: int64

In [11]:
#Dataframe with the new columns
df_driver = df_driver[new_columns]
#check for nulls
pd.isnull(df_driver).sum().sort_values(ascending=False)

AGE                              216583
VEHICLE_YEAR                     143139
FIRST_CONTACT_POINT                7320
LONGITUDE                          3197
LATITUDE                           3197
MODEL                              2707
MAKE                               2641
OCCUPANT_CNT                       2638
MANEUVER                           2638
VEHICLE_TYPE                       2638
VEHICLE_DEFECT                     2638
SEX                                  89
CRASH_TYPE                            0
ROAD_DEFECT                           0
ROADWAY_SURFACE_COND                  0
DAMAGE                                0
ALIGNMENT                             0
FIRST_CRASH_TYPE                      0
TRAFFICWAY_TYPE                       0
Crash_year                            0
LIGHTING_CONDITION                    0
SEC_CONTRIBUTORY_CAUSE                0
WEATHER_CONDITION                     0
DEVICE_CONDITION                      0
TRAFFIC_CONTROL_DEVICE                0


In [12]:
#dropping missing values
df_driver.dropna(axis=0, subset=['LATITUDE'], inplace =True)
df_driver.dropna(axis=0, subset=['LONGITUDE'], inplace =True)
df_driver.dropna(axis=0, subset=['MODEL'], inplace =True)
df_driver.dropna(axis=0, subset=['MAKE'], inplace =True)
df_driver.dropna(axis=0, subset=['FIRST_CONTACT_POINT'], inplace =True)

In [13]:
pd.isnull(df_driver).sum().sort_values(ascending=False).iloc[:5]

AGE                     213755
VEHICLE_YEAR            139208
SEX                         89
Crash_year                   0
ROADWAY_SURFACE_COND         0
dtype: int64

In [14]:
df_driver.INJURIES_FATAL.value_counts()

0.0    533762
1.0       457
2.0        27
3.0         4
4.0         1
Name: INJURIES_FATAL, dtype: int64

In [15]:
df_driver.AGE.value_counts()

25.0     10235
26.0     10012
24.0      9826
27.0      9809
28.0      9553
         ...  
104.0        2
107.0        1
109.0        1
110.0        1
108.0        1
Name: AGE, Length: 108, dtype: int64

In [16]:
df_driver.SEX.value_counts()

M    280127
F    160634
X     93401
Name: SEX, dtype: int64

In [17]:
df_driver.VEHICLE_YEAR.value_counts()

2015.0    27354
2016.0    25193
2014.0    24893
2017.0    23447
2013.0    23153
          ...  
2210.0        1
6043.0        1
6414.0        1
7007.0        1
2049.0        1
Name: VEHICLE_YEAR, Length: 150, dtype: int64

In [18]:
df_driver.VEHICLE_YEAR.describe()

count    395043.000000
mean       2015.429811
std         211.143007
min        1900.000000
25%        2005.000000
50%        2011.000000
75%        2015.000000
max        9999.000000
Name: VEHICLE_YEAR, dtype: float64

In [52]:
# Handling nulls for AGE, SEX, and VEHICLE_YEAR
df_driver = df_driver[(df_driver.VEHICLE_YEAR>=1970) & (df_driver.VEHICLE_YEAR <=2021)]
df_driver.AGE.replace({np.NAN: df_driver.AGE.median()},inplace=True)
df_driver.SEX.replace({np.NAN : 'X'},inplace=True)
# Reassigning values for INJURIES FATAL
df_driver.INJURIES_FATAL.replace({0.0:0, 1.0:1, 2.0:1, 3.0:1, 4.0:1}, inplace=True)
df_driver.shape

AttributeError: 'DataFrame' object has no attribute 'VEHICLE_YEAR'

In [20]:
df_driver.isna().sum()

CRASH_RECORD_ID                  0
CRASH_DATE_x                     0
POSTED_SPEED_LIMIT               0
TRAFFIC_CONTROL_DEVICE           0
DEVICE_CONDITION                 0
WEATHER_CONDITION                0
LIGHTING_CONDITION               0
FIRST_CRASH_TYPE                 0
TRAFFICWAY_TYPE                  0
ALIGNMENT                        0
ROADWAY_SURFACE_COND             0
ROAD_DEFECT                      0
CRASH_TYPE                       0
DAMAGE                           0
PRIM_CONTRIBUTORY_CAUSE          0
SEC_CONTRIBUTORY_CAUSE           0
NUM_UNITS                        0
MOST_SEVERE_INJURY               0
INJURIES_TOTAL                   0
INJURIES_FATAL                   0
INJURIES_INCAPACITATING          0
INJURIES_NON_INCAPACITATING      0
INJURIES_REPORTED_NOT_EVIDENT    0
CRASH_HOUR                       0
CRASH_DAY_OF_WEEK                0
CRASH_MONTH                      0
LATITUDE                         0
LONGITUDE                        0
MAKE                

In [21]:
# df_driver.hist(figsize=(20,20), bins='auto');

In [51]:
# dropping columns we do not need
columns_to_drop = ['CRASH_RECORD_ID','CRASH_DATE_x','INJURIES_TOTAL','INJURIES_INCAPACITATING',
                   'INJURIES_NON_INCAPACITATING','INJURIES_REPORTED_NOT_EVIDENT','MAKE','MOST_SEVERE_INJURY',
                   'MODEL','VEHICLE_YEAR','OCCUPANT_CNT','PERSON_TYPE',
                   'INJURY_CLASSIFICATION','CRASH_TYPE','LONGITUDE','LATITUDE','Crash_year']

df_driver = df_driver.drop(columns_to_drop,axis=1)

KeyError: "['CRASH_RECORD_ID' 'CRASH_DATE_x' 'INJURIES_TOTAL'\n 'INJURIES_INCAPACITATING' 'INJURIES_NON_INCAPACITATING'\n 'INJURIES_REPORTED_NOT_EVIDENT' 'MAKE' 'MOST_SEVERE_INJURY' 'MODEL'\n 'VEHICLE_YEAR' 'OCCUPANT_CNT' 'PERSON_TYPE' 'INJURY_CLASSIFICATION'\n 'CRASH_TYPE' 'LONGITUDE' 'LATITUDE' 'Crash_year'] not found in axis"

In [23]:
df_driver.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394153 entries, 0 to 2652389
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   POSTED_SPEED_LIMIT       394153 non-null  int64  
 1   TRAFFIC_CONTROL_DEVICE   394153 non-null  object 
 2   DEVICE_CONDITION         394153 non-null  object 
 3   WEATHER_CONDITION        394153 non-null  object 
 4   LIGHTING_CONDITION       394153 non-null  object 
 5   FIRST_CRASH_TYPE         394153 non-null  object 
 6   TRAFFICWAY_TYPE          394153 non-null  object 
 7   ALIGNMENT                394153 non-null  object 
 8   ROADWAY_SURFACE_COND     394153 non-null  object 
 9   ROAD_DEFECT              394153 non-null  object 
 10  DAMAGE                   394153 non-null  object 
 11  PRIM_CONTRIBUTORY_CAUSE  394153 non-null  object 
 12  SEC_CONTRIBUTORY_CAUSE   394153 non-null  object 
 13  NUM_UNITS                394153 non-null  int64  
 14  INJ

In [24]:
df_driver.describe()

,POSTED_SPEED_LIMIT,NUM_UNITS,INJURIES_FATAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,AGE
count,394153.000000,394153.000000,394153.000000,394153.000000,394153.000000,394153.000000,394153.000000
mean,28.536797,2.031767,0.001081,13.355872,4.145758,6.765112,38.446012
std,6.243285,0.466209,0.032858,5.363479,1.962164,3.388142,14.436103
min,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,30.000000,2.000000,0.000000,10.000000,2.000000,4.000000,29.000000
50%,30.000000,2.000000,0.000000,14.000000,4.000000,7.000000,36.000000
75%,30.000000,2.000000,0.000000,17.000000,6.000000,10.000000,46.000000
max,99.000000,18.000000,1.000000,23.000000,7.000000,12.000000,110.000000


In [25]:
df_driver.shape

(394153, 28)

In [26]:
df_driver.PRIM_CONTRIBUTORY_CAUSE.value_counts()

UNABLE TO DETERMINE                                                                 127296
FAILING TO YIELD RIGHT-OF-WAY                                                        49377
FOLLOWING TOO CLOSELY                                                                46436
FAILING TO REDUCE SPEED TO AVOID CRASH                                               19139
IMPROPER OVERTAKING/PASSING                                                          18903
IMPROPER BACKING                                                                     18574
NOT APPLICABLE                                                                       17290
IMPROPER LANE USAGE                                                                  15807
IMPROPER TURNING/NO SIGNAL                                                           14759
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  13333
DISREGARDING TRAFFIC SIGNALS                                                          7794

In [27]:
# Subsetting df where primary contributory cause is determined
df_driver = df_driver[df_driver.PRIM_CONTRIBUTORY_CAUSE != 'UNABLE TO DETERMINE']
df_driver.shape

(266857, 28)

In [28]:
df_driver.SEC_CONTRIBUTORY_CAUSE.value_counts()

NOT APPLICABLE                                                                      111303
UNABLE TO DETERMINE                                                                  57063
FAILING TO REDUCE SPEED TO AVOID CRASH                                               17921
FAILING TO YIELD RIGHT-OF-WAY                                                        12926
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  12566
FOLLOWING TOO CLOSELY                                                                11519
IMPROPER OVERTAKING/PASSING                                                           5831
IMPROPER LANE USAGE                                                                   5711
WEATHER                                                                               5196
IMPROPER TURNING/NO SIGNAL                                                            4341
IMPROPER BACKING                                                                      3517

In [29]:
# Subsetting df where Secondary contributory cause is determined and applicable
df_driver = df_driver[(df_driver.SEC_CONTRIBUTORY_CAUSE != 'UNABLE TO DETERMINE')]
df_driver = df_driver[(df_driver.SEC_CONTRIBUTORY_CAUSE != 'NOT APPLICABLE')]
df_driver.shape

(98491, 28)

# Modeling

In [30]:
df_driver.columns

Index(['POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'DAMAGE', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE',
       'NUM_UNITS', 'INJURIES_FATAL', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'MANEUVER',
       'FIRST_CONTACT_POINT', 'SEX', 'AGE', 'AIRBAG_DEPLOYED', 'EJECTION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION'],
      dtype='object')

In [31]:
# Features to try
# features = ['AGE', 'CRASH_HOUR','CRASH_DAY_OF_WEEK', 'POSTED_SPEED_LIMIT','PRIM_CONTRIBUTORY_CAUSE',
#            'WEATHER_CONDITION','LIGHTING_CONDITION','TRAFFIC_CONTROL_DEVICE','DRIVER_VISION',
#            'PHYSICAL_CONDITION','ROADWAY_SURFACE_COND','VEHICLE_DEFECT','EJECTION','INJURIES_FATAL']
# test_df = df_driver[features]

In [32]:
# Decided Features to use
features = ['AGE', 'CRASH_HOUR','CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'POSTED_SPEED_LIMIT',
           'WEATHER_CONDITION','LIGHTING_CONDITION','TRAFFIC_CONTROL_DEVICE',
           'PHYSICAL_CONDITION','VEHICLE_DEFECT','INJURIES_FATAL']
test_df = df_driver[features]

## Train-Test Split

In [33]:
X = test_df.drop('INJURIES_FATAL',axis=1)
y = test_df.INJURIES_FATAL

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
print(len(X_train), len(X_test), len(y_train), len(y_test))

73868 24623 73868 24623


## Decision Tree

#### Trying to use a pipeline

In [34]:
X_train.head()

,AGE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,TRAFFIC_CONTROL_DEVICE,PHYSICAL_CONDITION,VEHICLE_DEFECT
852348,51.0,11,7,2,35,CLEAR,DAYLIGHT,TRAFFIC SIGNAL,NORMAL,NONE
2480544,18.0,20,1,11,30,CLEAR,"DARKNESS, LIGHTED ROAD",NO CONTROLS,NORMAL,NONE
414394,36.0,19,2,2,30,CLEAR,"DARKNESS, LIGHTED ROAD",STOP SIGN/FLASHER,UNKNOWN,NONE
1544387,52.0,10,5,4,30,CLEAR,DAYLIGHT,TRAFFIC SIGNAL,NORMAL,NONE
2597147,24.0,10,7,9,25,CLEAR,DAYLIGHT,NO CONTROLS,NORMAL,NONE


In [35]:
subpipe_num = Pipeline(steps=[('ss', StandardScaler())])
subpipe_cat = Pipeline(steps=[('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [36]:
CT = ColumnTransformer(transformers=[('subpipe_num', subpipe_num, [0, 1, 2,3,4]),
                                         ('subpipe_cat', subpipe_cat, [5,6,7,8,9])],
                           remainder='passthrough')

In [37]:
imb_pipe = ImPipeline(steps=[('ct', CT),
                             ('sm', SMOTE(sampling_strategy='auto',random_state=42)),
                            ('rfc', DecisionTreeClassifier(random_state=42))])

In [38]:
imb_pipe.fit(X_train,y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('subpipe_num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4]),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  [5, 6, 7, 8, 9])])),
                ('sm', SMOTE(random_state=42)),
                ('rfc', DecisionTreeClassifier(random_state=42))])

In [39]:
imb_pipe.score(X_train, y_train)

0.9999593870146748

In [40]:
imb_pipe.score(X_test, y_test)

0.9965885554156683

#### Not using pipeline

### One Hot Encoding 

In [41]:
#Ohe-ing Train set
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums_train = ohe.fit_transform(X_train_cat)
dums_train_df = pd.DataFrame(dums_train,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

nums_train_df = X_train.select_dtypes(['int64','float64'])

X_train_clean = pd.concat([nums_train_df, dums_train_df], axis=1)

In [42]:
#Ohe-ing Test set
X_test_cat = X_test.select_dtypes('object')

dums_test = ohe.transform(X_test_cat)
dums_test_df = pd.DataFrame(dums_test,
                       columns=ohe.get_feature_names(),
                       index=X_test_cat.index)

nums_test_df = X_test.select_dtypes(['int64','float64'])

X_test_clean = pd.concat([nums_test_df, dums_test_df], axis=1)

ValueError: Found unknown categories ['EXHAUST'] in column 4 during transform

### Smote

In [43]:
# X_train_clean.columns
smote = SMOTE(sampling_strategy='auto',random_state=42)
X_train_resampled, y_train_resampled = smote.fit_sample(X_train_clean, y_train) 

In [44]:
X_train_resampled.shape

(147534, 64)

In [45]:
y_train.value_counts()

0.0    73767
1.0      101
Name: INJURIES_FATAL, dtype: int64

In [46]:
y_train_resampled.value_counts()

1.0    73767
0.0    73767
Name: INJURIES_FATAL, dtype: int64

### Baseline (Vanilla) Decision Tree

In [55]:
baseline_tree = DecisionTreeClassifier(random_state=42)
baseline_tree.fit(X_train_resampled,y_train_resampled)
baseline_tree_train_cv = cross_val_score(baseline_tree, X_train_resampled, y_train_resampled).mean()
baseline_tree_test_cv = cross_val_score(baseline_tree, X_test_clean, y_test).mean()

print(f'DT baseline Train Cross Validation Score: {baseline_tree_train_cv}')
print(f'DT baseline Test Cross Validation Score: {baseline_tree_test_cv}')

DT baseline Train Cross Validation Score: 0.9979467731878373
DT baseline Test Cross Validation Score: 0.9966703392956878


In [56]:
# fig, ax = plt.subplots(figsize=(20,20))
# plot_tree(baseline_tree,ax=ax);

### Finding Best Decision Tree Model

In [47]:
tree = DecisionTreeClassifier(random_state=42)

In [48]:
tree_grid = {'max_leaf_nodes': list(range(2, 100)), 
             'min_samples_split': [2, 3, 4],
             'max_depth': [5, 6, 7, 8, 9],
            }

In [49]:
tree_grid_search = GridSearchCV(tree, tree_grid, cv=5)

In [50]:
tree_grid_search.fit(X_train_resampled, y_train_resampled)

KeyboardInterrupt: 

In [ ]:
tree_grid_search.best_params_

In [ ]:
tree_grid_search.best_score_

In [ ]:
tree_grid_search.best_estimator_.score(X_test_clean, y_test)

In [ ]:
tree_cv_df = pd.DataFrame(tree_grid_search.cv_results_)
tree_cv_df.head()

In [ ]:
best_tree = tree_grid_search.best_estimator_

In [ ]:
def plot_feature_importances(model):
    n_features = X_train_resampled.shape[1]
    plt.figure(figsize=(30,30))
    plt.barh(range(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), X_train_resampled.columns.values) 
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

In [ ]:
plot_feature_importances(best_tree)

# Random Forest

### Baseline (Vanilla) Random Forest

In [ ]:
baseline_forest = RandomForestClassifier(random_state=42)
baseline_forest.fit(X_train_resampled,y_train_resampled)
baseline_forest_train_cv = cross_val_score(baseline_forest, X_train_resampled, y_train_resampled).mean()
baseline_forest_test_cv = cross_val_score(baseline_forest, X_test_clean, y_test).mean()

print(f'DT baseline Train Cross Validation Score: {baseline_forest_train_cv}')
print(f'DT baseline Test Cross Validation Score: {baseline_forest_test_cv}')

### Finding Best Random Forest Model

In [ ]:
forest = RandomForestClassifier(random_state=42)

In [ ]:
# Number of trees in random forest
# n_estimators = [100,150,200]

# The number of features to consider when looking for the best split
max_features = ['sqrt', 'log2']

# function to measure the quality of a split
criteria = ['gini', 'entropy']

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Create the random grid
forrest_grid = {'max_features': max_features, 
                'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf
              }

In [ ]:
forrest_grid_search = GridSearchCV(estimator=forest, param_grid=forrest_grid, cv=5)

In [ ]:
forrest_grid_search.fit(X_train_resampled, y_train_resampled)

In [ ]:
forrest_grid_search.best_params_

In [ ]:
forrest_grid_search.best_score_

In [ ]:
forrest_grid_search.best_estimator_.score(X_test_clean, y_test)

In [ ]:
best_forest = forrest_grid_search.best_estimator_

In [ ]:
plot_feature_importances(best_forest)

In [ ]:
forest_cv_df = pd.DataFrame(forrest_grid_search.cv_results_)
forest_cv_df.head()

# K Nearest Neighbor

In [ ]:
# Splitting categorical and numeric features from train and test set
X_train_cat = X_train.select_dtypes('object')
X_test_cat = X_test.select_dtypes('object')
nums_train = X_train.select_dtypes(['int64','float64'])
nums_test = X_test.select_dtypes(['int64','float64'])

# One Hot Encoding the categoricals
ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums_train = ohe.fit_transform(X_train_cat)
dums_train_df = pd.DataFrame(dums_train,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

dums_test = ohe.transform(X_test_cat)
dums_test_df = pd.DataFrame(dums_test,
                       columns=ohe.get_feature_names(),
                       index=X_test_cat.index)
# Scaling the Numerics
scaler = StandardScaler()

nums_train_scaled = scaler.fit_transform(nums_train)
nums_train_df = pd.DataFrame(nums_train_scaled,
                       columns=nums_train.columns,
                       index=nums_train.index)

nums_test_scaled = scaler.transform(nums_test)
nums_test_df = pd.DataFrame(nums_test_scaled,
                       columns=nums_test.columns,
                       index=nums_test.index)

# Scaled and Encoded train and test data
X_train_scaled = pd.concat([nums_train_df, dums_train_df], axis=1)
X_test_scaled = pd.concat([nums_test_df, dums_test_df], axis=1)

# SMOTE
smote = SMOTE(sampling_strategy='auto',random_state=42)
X_train_resampled, y_train_resampled = smote.fit_sample(X_train_scaled, y_train) 

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
# create params_grid for KNeighborsClassifier  
knn_grid = {'n_neighbors': [3, 9, 11],
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan']}

# create grid search 
knn_grid = GridSearchCV(knn, knn_grid, cv=5)

In [ ]:
# fit x_train and y_train to grid 
knn_grid.fit(X_train_resampled,y_train_resampled)

In [ ]:
# observe combination of best params 
knn_grid.best_params_

In [ ]:
knn_cv_df = pd.DataFrame(knn_grid.cv_results_)
knn_cv_df.head()

In [ ]:
knn_grid.best_estimator_.score(X_test_scaled, y_test)

In [ ]:
test_preds = knn_grid.best_estimator_.predict(X_test_scaled)